In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
train_data_dir = 'train'
validation_data_dir = 'val'

In [3]:
# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Preprocess and augment validation data
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [4]:
batch_size = 32
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Input size for VGG16
    batch_size=batch_size,
    class_mode='binary'  # 'binary' for two classes: fire and not fire
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

Found 1887 images belonging to 2 classes.
Found 402 images belonging to 2 classes.


In [5]:
# Load the VGG16 model, excluding the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model with a custom classification head
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # 1 neuron for binary classification

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
epochs = 1
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

36/58 [=================>............] - ETA: 6:54 - loss: 0.6169 - accuracy: 0.6603

c:\Users\Charlie\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


54/58 [==========================>...] - ETA: 1:12 - loss: 0.5829 - accuracy: 0.6885

c:\Users\Charlie\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


58/58 [==============================] - 1201s 21s/step - loss: 0.5692 - accuracy: 0.6976 - val_loss: 0.5385 - val_accuracy: 0.7057


In [7]:
# Evaluate the model
score = model.evaluate(validation_generator)
print("Validation Loss: {:.2f}".format(score[0]))
print("Validation Accuracy: {:.2f}".format(score[1]))

# Save the model
model.save('fire_detection_model.h5')

13/13 [==============================] - 98s 7s/step - loss: 0.5303 - accuracy: 0.7139
Validation Loss: 0.53
Validation Accuracy: 0.71


In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [10]:
model = load_model('fire_detection_model.h5')

img_path = 'fire.jpg'
img = image.load_img(img_path, target_size=(224, 224))  # Target size should match the model's input size

img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize pixel values

# Reshape the image to match the input shape of the model
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)

class_indices = {0: "fire", 1: "not fire"}  # Map class indices to class labels

predicted_class_index = np.argmax(predictions)
predicted_class = class_indices[predicted_class_index]

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 259ms/step
Predicted class: fire
